<a href="https://colab.research.google.com/github/shirsh008/reinforcement-learning-model-training/blob/main/milp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
  data = {}
  data["time_matrix"] = [
        [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
        [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
        [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
        [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
        [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
        [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
        [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
        [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
        [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
        [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
        [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
        [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
        [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
        [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
        [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
        [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
        [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
    ]
  data["time_windows"] = [
        (0, 5),  # depot
        (7, 12),  # 1
        (10, 15),  # 2
        (5, 14),  # 3
        (5, 13),  # 4
        (0, 5),  # 5
        (5, 10),  # 6
        (0, 10),  # 7
        (5, 10),  # 8
        (0, 5),  # 9
        (10, 16),  # 10
        (10, 15),  # 11
        (0, 5),  # 12
        (5, 10),  # 13
        (7, 12),  # 14
        (10, 15),  # 15
        (5, 15),  # 16
    ]
  data["num_vehicles"] = 4
  data["vehicle_load_time"] = 5
  data["vehicle_unload_time"] = 5
  data["depot_capacity"] = 2
  data["depot"] = 0
  return data

def print_solution(data, manager, routing, solution):
  print("Objective Value : ", solution.ObjectiveValue())
  time_dimension = routing.GetDimensionOrDie("Time")
  total_time = 0
  for vehicle_id in range(data["num_vehicles"]):
    if not routing.IsVehicleUsed(solution, vehicle_id):
      continue
    index = routing.Start(vehicle_id)
    plan_output = "Route for vehicle {}:\n".format(vehicle_id)
    while not routing.IsEnd(index):
      time_var = time_dimension.CumulVar(index)
      plan_output += (
          f"{manager.IndexToNode(index)}"
          f" Time({solution.Min(time_var)}, {solution.Max(time_var)})"
          " -> "
      )
      index = solution.Value(routing.NextVar(index))
    time_var = time_dimension.CumulVar(index)
    plan_output += (
        f"{manager.IndexToNode(index)}"
        f" Time({solution.Min(time_var)}, {solution.Max(time_var)})\n"
    )
    plan_output += f"Time of the route : {solution.Min(time_var)}min\n"
    print(plan_output)
    total_time += solution.Min(time_var)
  print(f"Total time of all routes : {total_time}min")

def main():
  data = create_data_model()
  manager = pywrapcp.RoutingIndexManager(
      len(data["time_matrix"]), data["num_vehicles"], data["depot"]
  )

  routing = pywrapcp.RoutingModel(manager)

  def time_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data["time_matrix"][from_node][to_node]

  transit_callback_index = routing.RegisterTransitCallback(time_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  time = "Time"
  routing.AddDimension(
      transit_callback_index,
      60,
      60,
      False,
      time,
  )
  time_dimension = routing.GetDimensionOrDie(time)

  for location_idx, time_window in enumerate(data["time_windows"]):
    if location_idx == 0:
      continue
    index = manager.IndexToNode(location_idx)
    time_dimension.CumulVar(index).SetRange(
        time_window[0], time_window[1]
    )
  for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(
        data["time_windows"][0][0], data["time_windows"][0][1]
    )

  solver = routing.solver()
  intervals = []
  for i in range(data["num_vehicles"]):
    intervals.append(
        solver.FixedDurationIntervalVar(
            time_dimension.CumulVar(routing.Start(i)),
            data["vehicle_load_time"],
            "depot_interval",
        )
    )
    intervals.append(
        solver.FixedDurationIntervalVar(
            time_dimension.CumulVar(routing.End(i)),
            data["vehicle_unload_time"],
            "depot_interval",
        )
    )

  depot_usage = [1 for _ in range(len(intervals))]
  solver.Add(
      solver.Cumulative(
          intervals, depot_usage, data["depot_capacity"], "depot"
      )
  )

  for i in range(data["num_vehicles"]):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i))
    )
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i))
    )

  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (
      routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
  )

  solution = routing.SolveWithParameters(search_parameters)
  if solution:
    print_solution(data, manager, routing, solution)
  else:
    print("No solution")

if __name__ == "__main__":
  main()

Objective Value :  71
Route for vehicle 0:
0 Time(5, 5) -> 8 Time(8, 9) -> 14 Time(11, 12) -> 16 Time(13, 15) -> 0 Time(25, 25)
Time of the route : 25min

Route for vehicle 1:
0 Time(0, 0) -> 12 Time(4, 4) -> 13 Time(6, 6) -> 15 Time(11, 11) -> 11 Time(14, 14) -> 0 Time(20, 20)
Time of the route : 20min

Route for vehicle 2:
0 Time(5, 5) -> 7 Time(7, 7) -> 1 Time(11, 11) -> 4 Time(13, 13) -> 3 Time(14, 14) -> 0 Time(24, 24)
Time of the route : 24min

Route for vehicle 3:
0 Time(0, 0) -> 9 Time(2, 3) -> 5 Time(4, 5) -> 6 Time(6, 9) -> 2 Time(10, 12) -> 10 Time(14, 16) -> 0 Time(29, 29)
Time of the route : 29min

Total time of all routes : 98min
